<a href="https://colab.research.google.com/github/SAWGraph/public/blob/main/UseCases/UC1-Testing/UC1-CQ2a/UC1_CQ2a_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

The purpose of this notebook is to query the SAWGraph knowledge graph for landfills in Penobscot County, Maine, that are near any water body.

This is a demonstration of competency question 2a from Use Case 1: Testing.

- Retrieve all landfills in Penobscot county that are near (have touching S2 cell) any waterbody.

# Setup

Here we set up SPARQLWrapper to work with our endpoint and create our query.

## Install & Import Statements

Install: The *SPARQLWrapper* libary provides tools for querying SPARQL endpoints. The *sparql_dataframe* library can be used with SPARQLWrapper to convert JSON results from a SPARQL query directly to a Pandas dataframe. The *mapclassify* library is required by GeoPandas for its *.explore* functionality.

Import: See the inline comments for a brief rationale of each library.

In [2]:
%%capture
!pip install mapclassify --upgrade --quiet
!pip install SPARQLWrapper --upgrade --quiet
!pip install sparql_dataframe --upgrade --quiet

In [3]:
from branca.element import Figure                                  # For controlling the size of the final map
import folium                                                      # For map layer control
import geopandas as gpd                                            # For geospatial dataframes
import pandas as pd                                                # For dataframes
from shapely import wkt                                            # For working with WKT coordinates in a GeoDataFrame
from SPARQLWrapper import SPARQLWrapper, JSON, GET, POST, DIGEST   # For querying SPARQL endpoints
import sparql_dataframe                                            # For converting SPARQL query results to Pandas dataframes
from tabulate import tabulate                                      # For pretty printing dataframes

## Variable Initialization

A SPARQLWrapper is created for each required repository.

In [4]:
%%capture
pd.options.display.width = 240

endpointFIO = 'https://gdb.acg.maine.edu:7200/repositories/FIO'
sparqlFIO = SPARQLWrapper(endpointFIO)
sparqlFIO.setHTTPAuth(DIGEST)
sparqlFIO.setCredentials('sawgraph-endpoint', 'skailab')
sparqlFIO.setMethod(GET)
sparqlFIO.setReturnFormat(JSON)

endpointS2L13AdminRegions = 'https://gdb.acg.maine.edu:7200/repositories/S2L13_AdminRegions'
sparqlS2L13AdminRegions = SPARQLWrapper(endpointS2L13AdminRegions)
sparqlS2L13AdminRegions.setHTTPAuth(DIGEST)
sparqlS2L13AdminRegions.setCredentials('sawgraph-endpoint', 'skailab')
sparqlS2L13AdminRegions.setMethod(GET)
sparqlS2L13AdminRegions.setReturnFormat(JSON)

endpointHydrology = 'https://gdb.acg.maine.edu:7200/repositories/Hydrology'
sparqlHydrology = SPARQLWrapper(endpointHydrology)
sparqlHydrology.setHTTPAuth(DIGEST)
sparqlHydrology.setCredentials('sawgraph-endpoint', 'skailab')
sparqlHydrology.setMethod(GET)
sparqlHydrology.setReturnFormat(JSON)

## Query

This query access the FIO repository directly and uses federation to access data in the S2L13_AdminRegions and Hydrology repositories.

The first block retrieves landfills and the S2 cells they are within. It pulls label (facility name) and industry information if it is available.

The second block finds S2 cells that neighbor (`kwg-ont:sfTouch`) the landfill S2 cells found in the third block and it returns the landfill S2 cell(s). (`owl:sameAs`). It also restricts the spatial coverage to Penobscot County, Maine.

The third block selects water bodies that overlap any of the S2 cells from the second block.

The query results are returned as a dataframe.


In [5]:
%%time
query = """
PREFIX fio: <http://sawgraph.spatialai.org/v1/fio#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX naics: <http://sawgraph.spatialai.org/v1/fio/naics#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>

SELECT * WHERE {
    ?fac rdf:type fio:Facility ;
            fio:ofIndustry ?code ;
            kwg-ont:sfWithin ?fac_s2 ;
            kwg-ont:sfWithin ?county .
    OPTIONAL { ?fac rdfs:label ?faclabel . }
    OPTIONAL { ?code rdfs:label ?ind . }
    FILTER (?code IN (naics:NAICS-IndustryCode-562212))

    SERVICE <repository:S2L13_AdminRegions> {
        SELECT * WHERE {
            ?fac_s2 rdf:type kwg-ont:S2Cell_Level13 ;
            		kwg-ont:sfTouches | owl:sameAs ?nbr_s2 .
            ?county rdf:type kwg-ont:AdministrativeRegion_2 ;
                    rdfs:label ?county_label .
            FILTER regex(?county_label, "penobscot", "i")
        }
    }

    SERVICE <repository:Hydrology> {
        SELECT * WHERE {
            ?wb rdf:type hyf:HY_WaterBody ;
                rdfs:label ?wblabel .
            ?nbr_s2 kwg-ont:sfOverlaps ?wb .
        }
    }
}
"""
df = sparql_dataframe.get(endpointFIO, query)

CPU times: user 319 ms, sys: 32 ms, total: 351 ms
Wall time: 1min


# Geometry Queries

This is a function to create a query for geometries for a list of instances in a KG.

In [6]:
def geo_query(data):
    instances = ''
    for d in data:
        instances += '<' + d + '>, '
    instances = instances[:-2]
    query = """
    PREFIX geo: <http://www.opengis.net/ont/geosparql#>

    SELECT * WHERE {
        ?x geo:hasGeometry/geo:asWKT ?wkt .
        FILTER (?x IN (""" + instances + """))
    }
    """
    return query

The SPARQL endpoint can only process a limited number of instances at a time (at least 100). The following function determines the number of instances that require geometries and retrieves them.

In [7]:
def retrieve_geometries(instances, repo):
    if len(instances) < 101:
        query = geo_query(instances)
        return sparql_dataframe.get(repo, query)
    else:
        data_dict = {}
        for i in range(len(instances) // 100 + 1):
            data_dict[i] = instances[i * 100:(i + 1) * 100]
        df_dict = {}
        for k, v in data_dict.items():
            query = geo_query(v)
            df_temp = sparql_dataframe.get(repo, query)
            if k == 0:
                df_dict[k] = [df_temp.columns.values.tolist()] + df_temp.values.tolist()
            else:
                df_dict[k] = df_temp.values.tolist()
        data_list = []
        for k, v in df_dict.items():
            for item in v:
                data_list.append(item)
        df_geo = pd.DataFrame(data_list[1:], columns=data_list[0])
        df_geo.drop_duplicates(inplace=True)
        return df_geo

Create lists of facilities, facility S2 cells, neighboring S2 cells, and water bodies.

In [8]:
county = []
fac = []
fac_s2 = []
nbr_s2 = []
wb = []
for row in df.itertuples():
    if row.county not in county:
        county.append(row.county)
    if row.fac not in fac:
        fac.append(row.fac)
    if row.fac_s2 not in fac_s2:
        fac_s2.append(row.fac_s2)
    if row.nbr_s2 not in nbr_s2:
        nbr_s2.append(row.nbr_s2)
    if row.wb not in wb:
        wb.append(row.wb)

Retreive geometries for the instances in the above lists.

In [9]:
%%time
df_county_geo = retrieve_geometries(county, endpointS2L13AdminRegions)
df_fac_geo = retrieve_geometries(fac, endpointFIO)
df_fac_s2_geo = retrieve_geometries(fac_s2, endpointS2L13AdminRegions)
df_nbr_s2_geo = retrieve_geometries(nbr_s2, endpointS2L13AdminRegions)
df_wb_geo = retrieve_geometries(wb, endpointHydrology)

CPU times: user 58.5 ms, sys: 4.16 ms, total: 62.7 ms
Wall time: 1.92 s


We have to add the geometries to the primary query's dataframe of results. This is done using the following function, which is called four times in this case for four different geometry types (facilities, facility S2 cells, neighboring S2 cells, and water bodies).

In [10]:
def new_column(df_in, df_wkt, col_item, col_wkt):
    df_out = df_in.copy()
    df_out[col_wkt] = df_out[col_item]
    for row in df_wkt.itertuples():
        df_out.loc[df_out[col_wkt] == row.x, col_wkt] = row.wkt
    return df_out

In [11]:
df = new_column(df, df_county_geo, 'county', 'county_wkt')
df = new_column(df, df_fac_geo, 'fac', 'fac_wkt')
df = new_column(df, df_fac_s2_geo, 'fac_s2', 'fac_s2_wkt')
df = new_column(df, df_nbr_s2_geo, 'nbr_s2', 'nbr_s2_wkt')
df = new_column(df, df_wb_geo, 'wb', 'wb_wkt')

# Visualizing on a map

## Partioning the Query Results

Here the data is split into four separate themed dataframes, one for the facilities, one for the facility S2 cells, one for the neighboring S2 cells, and one for the water bodies.

The columns with WKT coordinates in each dataframe are converted to Shapely geometry objects prior to converting the dataframes to GeoDataFrames.

In [12]:
df_county = df[['county', 'county_label', 'county_wkt']].copy()
df_county.drop_duplicates(inplace=True)
df_county['county_wkt'] = df_county['county_wkt'].apply(wkt.loads)

df_fac = df[['fac', 'faclabel', 'fac_wkt']].copy()
df_fac.drop_duplicates(inplace=True)
df_fac['fac_wkt'] = df_fac['fac_wkt'].apply(wkt.loads)

df_fac_s2 = df[['fac_s2', 'fac_s2_wkt']].copy()
df_fac_s2.drop_duplicates(inplace=True)
df_fac_s2['fac_s2_wkt'] = df_fac_s2['fac_s2_wkt'].apply(wkt.loads)

df_nbr_s2 = df[['nbr_s2', 'nbr_s2_wkt']].copy()
df_nbr_s2.drop_duplicates(inplace=True)
df_nbr_s2['nbr_s2_wkt'] = df_nbr_s2['nbr_s2_wkt'].apply(wkt.loads)

df_wb = df[['wb', 'wblabel', 'wb_wkt']].copy()
df_wb.drop_duplicates(inplace=True)
df_wb['wb_wkt'] = df_wb['wb_wkt'].apply(wkt.loads)

## Create GeoPandas dataframes for mapping

Convert the above dataframes to GeoDataFrames, setting the WKT columns as the geometry columns and setting the CRS to WGS 84.

In [13]:
%%capture
gdf_county = gpd.GeoDataFrame(df_county, geometry='county_wkt')
gdf_county.set_crs(epsg=4326, inplace=True, allow_override=True)

gdf_fac = gpd.GeoDataFrame(df_fac, geometry='fac_wkt')
gdf_fac.set_crs(epsg=4326, inplace=True, allow_override=True)

gdf_fac_s2 = gpd.GeoDataFrame(df_fac_s2, geometry='fac_s2_wkt')
gdf_fac_s2.set_crs(epsg=4326, inplace=True, allow_override=True)

gdf_nbr_s2 = gpd.GeoDataFrame(df_nbr_s2, geometry='nbr_s2_wkt')
gdf_nbr_s2.set_crs(epsg=4326, inplace=True, allow_override=True)

gdf_wb = gpd.GeoDataFrame(df_wb, geometry='wb_wkt')
gdf_wb.set_crs(epsg=4326, inplace=True, allow_override=True)

## Create map with multiple layers

Each GeoDataFrame is a layer in the final map.

In [14]:
%%capture
county_color = 'green'
fac_color = 'red'
fac_s2_color = 'darkred'
s2_color = 'black'
wb_color = 'blue'
boundweight = 5

map = gdf_county.explore(color=county_color,
                         style_kwds=dict(weight=boundweight),
                         tooltip=True,
                         name='<span style="color: ' + county_color + ';">County</span>',
                         fill=False,
                         show=True)
gdf_fac.explore(m=map,
                color=fac_color,
                style_kwds=dict(weight=boundweight),
                tooltip=True,
                name='<span style="color: ' + fac_color + ';">Facilities</span>',
                show=True)
gdf_fac_s2.explore(m=map,
                   color=fac_s2_color,
                   style_kwds=dict(weight=boundweight),
                   tooltip=True,
                   name='<span style="color: ' + fac_s2_color + ';">Facilities S2 Cells</span>',
                   show=False)
gdf_nbr_s2.explore(m=map,
                   color=s2_color,
                   style_kwds=dict(weight=boundweight),
                   tooltip=True,
                   name='<span style="color: ' + s2_color + ';">S2 Cell Neighbors</span>',
                   show=False)
gdf_wb.explore(m=map,
               color=wb_color,
               style_kwds=dict(weight=boundweight),
               tooltip=True,
               highlight=False,
               name='<span style="color: ' + wb_color + ';">Water Bodies</span>',
               show=True)

# folium.TileLayer("stamenterrain", show=False).add_to(map)
# folium.TileLayer("MapQuest Open Aerial", show=False).add_to(map)
folium.LayerControl(collapsed=False).add_to(map)

## Show map

The map is created inside a Figure box to control its size.

In [23]:
map.save('SAWGraph_UC1_CQ2a_map.html')

fig = Figure(width=800, height=600)
fig.add_child(map)

# Looking Further

Once the results of the primary query are available, it is possible to query for additional information related to specific results.

Here you can view some basic information for water bodies. Select a water body from the drop down and then press the 'play' button on the left. Not all water bodies have names. This data comes from the National Hydrography Database Plus Version 2 (NHDPlus v2).

In [16]:
water_body = 'comid/1699482' #@param ['comid/1699482', 'comid/1735118', 'comid/2680590', 'comid/2680638']
query = """
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT * WHERE {
    SERVICE <repository:Hydrology> {
        SELECT * WHERE {
            ?wb rdf:type hyf:HY_WaterBody ;
                rdfs:label ?wblabel ;
                rdfs:comment ?wbcomment .
            FILTER(?wb = <https://geoconnex.us/nhdplusv2/""" + water_body + """>)
        }
    }
}
"""
df2 = sparql_dataframe.get(endpointHydrology, query)
# print(tabulate(df2, headers=['Water body instance', 'Water body name', 'Comments'], tablefmt='rounded_outline', showindex=False))
print(f'\nBasic info for {water_body}\n')
wb = ''; wb_name = ''; comments = []
for row in df2.itertuples():
    wb = row.wb
    wb_name = row.wblabel
    comments.append(row.wbcomment)
print(f'Water body instance: {wb}')
print(f'   {wb_name}')
for comment in comments:
    print(f'   {comment}')
print()


Basic info for comid/1699482

Water body instance: https://geoconnex.us/nhdplusv2/comid/1699482
   GNIS_NAME: Dolby Pond
   FTYPE: LakePond
   COMID: 1699482
   Reachcode: 01020001001834



Facility information comes from the Facility Registry Service (FRS) from the Environmental Protection Agency (EPA). Select a facility from the drop down, press the 'play' button on the left, and you will get some basic information for each of the available facilities. A single facility can be classified in more than one industry.

In [22]:
facility = 'FRS-Facility.110055618577' #@param ['FRS-Facility.110032749177', 'FRS-Facility.110039664342', 'FRS-Facility.110055618577', 'FRS-Facility.110058407941']
query = """
PREFIX fio: <http://sawgraph.spatialai.org/v1/fio#>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX us_frs: <http://sawgraph.spatialai.org/v1/us-frs#>

SELECT ?fac ?fac_name ?admin_region_name ?primary_industry ?fac_frs_id WHERE {
    SERVICE <repository:FIO> {
        SELECT * WHERE {
            ?fac rdf:type fio:Facility ;
                 rdfs:label ?fac_name ;
                 us_frs:primaryIndustry/rdfs:label ?primary_industry ;
                 us_frs:hasFRSId ?fac_frs_id ;
                 kwg-ont:sfWithin ?admin_region .
            FILTER(?fac = <http://sawgraph.spatialai.org/v1/us-frs-data#d.""" + facility + """>)
        }
    }

    SERVICE <repository:S2L13_AdminRegions> {
        SELECT * WHERE {
            ?admin_region rdf:type kwg-ont:AdministrativeRegion_3 .
            ?admin_region rdfs:label ?admin_region_name .
        }
    }
}
"""
df3 = sparql_dataframe.get(endpointHydrology, query).fillna('--')
# print(tabulate(df3, headers=['Facility instance', 'Facility Name', 'Facility location', 'Primary Industry', 'FRS ID'], tablefmt='fancy_outline', showindex=False))
print(f'\nBasic info for {facility}\n')
fac = ''; fac_name = ''; fac_loc = ''; industries = []; frs_id = ''
for row in df3.itertuples():
    fac = row.fac
    fac_name = row.fac_name
    fac_loc = row.admin_region_name
    industries.append(row.primary_industry)
    frs_id = row.fac_frs_id
print(f'Facility instance: {fac}')
print(f'   Facility name: {fac_name}')
print(f'   Facility town: {fac_loc}')
for industry in industries:
    print(f'   Primary industry: {industry}')
print(f'   FRS ID: {frs_id}')
print()


Basic info for FRS-Facility.110055618577

Facility instance: http://sawgraph.spatialai.org/v1/us-frs-data#d.FRS-Facility.110055618577
   Facility name: ORONO WATER POLLUTION CONTROL FACILITY
   Facility town: Orono town, Maine
   Primary industry: Sewage Treatment Facilities 
   Primary industry: Solid Waste Landfill 
   FRS ID: 110055618577

